In [ ]:
!pip install objectnat

In [ ]:
!pip install pyarrow 
#To read .parquet

In [ ]:
from objectnat import get_adj_matrix_gdf_to_gdf, get_intermodal_graph
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union

buildings = gpd.read_parquet('examples_data/buildings.parquet')
services = gpd.read_parquet('examples_data/services.parquet')

polygon = unary_union([buildings.geometry.to_list() + services.geometry.to_list()]).convex_hull.buffer(
    0.001)  # Getting an "encircling" polygon around our layer to download the graph 
G_intermodal = get_intermodal_graph(polygon=polygon, clip_by_bounds=True)

buildings.to_crs(G_intermodal.graph['crs'], inplace=True)
services.to_crs(G_intermodal.graph['crs'], inplace=True)

In [ ]:
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(gdf_from=buildings,
                                                 gft_to=services,
                                                 nx_graph=G_intermodal,
                                                 weight="time_min",
                                                 )

In [ ]:
matrix.to_parquet('examples_data/matrix_time.parquet')